# 9.1题求解过程

In [ ]:
import chapter_9_module as c9m

a = 2
b = 0.6
t = 0.012
q = 6.5 / 0.01 / 0.01
sigma_0 = 18.8e6

## 先验证是否可使用板条梁模型（过程略）

## 求解（a）：
1. 计算 A 点弯矩和应力
$$M_A = \frac{q b^2}{24}$$
$$\sigma_A = \frac{6 M}{t^2}$$


In [ ]:
M_A = c9m.bendingMomentFixedEnd1(q, b)
print(f"A点弯矩 M_A 为 {M_A :.2f} N*m")
sigma_A = c9m.bendingNormalStress(M_A, t)
print(f"A点由弯曲引起的正应力 sigma_A 为 {sigma_A*1e-6 :.2f} MPa")

2. 计算 B 点弯矩和应力  
$$M_B = \frac{q b^2}{12}$$
$$\sigma_B = \frac{6 M_B}{t^2}$$

In [ ]:
M_B = c9m.bendingMomentFixedEnd2(q, b)
print(f"B点弯矩 M_B 为 {M_B :.2f} N*m")
sigma_B = c9m.bendingNormalStress(M_B, t)
print(f"B点由弯曲引起的正应力 sigma_B 为 {sigma_B*1e-6 :.2f} MPa")

3. 计算 A 点挠度  
$$w_A = \frac{q b^4}{384 E_1 I} = \frac{q b^4}{384 D}$$

In [ ]:
w_A = c9m.midSpanDeflection1(q, b, c9m.E1(), t)
print(f"A点挠度 w_A 为 {w_A*1e3 :.3f} mm")

## 求解（b）：
注：$w_{A} < \frac{t}{5}$，因此为小挠度问题
1. 计算板条梁中面拉力  
$$T = \sigma_0 \times 1 \times t$$

In [ ]:
T = 18.8e6 * 1 * t
print(f"板条梁中面拉力 T 为 {T :.2f} N")

2. 计算系数 u  
$$u = \frac{l}{2} \sqrt \frac{\sigma_0 t}{D} = \frac{l}{2 t} \sqrt \frac{12(1-\mu^2) \sigma_0}{E}$$

In [ ]:
u = c9m.u(b, T, c9m.cylindricalStiffness(t))
print(f"u = {u :.3f}")

3. 计算 A 点辅助函数、弯矩、正应力  
$$M{_A}' = M_A \phi_1 (u)$$
$$\phi_1 (u) = \frac{6}{u^2} (1 - \frac{u}{sinh(u)})$$
$$\sigma{_{A,b}} = \frac{6M{{_A}'}}{t^2}$$
$$\sigma{_{A,max}} = \sigma_0 + \sigma{_{A,b}}$$
$$w_{A}' = w_Af_1(u)$$
$$f_1(u) = \frac{24}{u^3}(\frac{u}{2} - tanh(\frac{u}{2}))$$

In [ ]:
print(f"phai1(u) = {c9m.phai1(u) :.3f}")
M_A_complex = M_A * c9m.phai1(u)
print(f"A点由复杂弯曲引起的弯矩 M_A_complex 为 {M_A_complex :.2f} N*m")
sigma_A_complex = c9m.bendingNormalStress(M_A_complex, t)
print(
    f"A点由复杂弯曲引起的正应力 sigma_A_complex 为 {(sigma_0+sigma_A_complex)*1e-6 :.2f} MPa"
)
print(f"f1(u) = {c9m.f1(u) :.3f}")
w_A_complex = w_A * c9m.f1(u)
print(f"A点复杂弯曲挠度 w_A_complex 为 {w_A_complex*1e3 :.3f} mm")

4. 计算 B 点弯矩和应力
$$M_{B}' = M_B \chi(u)$$
$$\chi(u) = \frac{3}{u^2} (\frac{u}{tanh(u)}-1)$$
$$\sigma{_{B,b}} = \frac{6M_{B}'}{t^2}$$
$$\sigma{_{B,max}} = \sigma_0 + \sigma{_{B,b}}$$

In [ ]:
print(f"X(u) = {c9m.X(u) :.3f}")
M_B_complex = c9m.bendingMomentFixedEnd2(q, b) * c9m.X(u)
print(f"B点由复杂弯曲引起的弯矩 M_B_complex 为 {M_B_complex :.2f} N*m")
sigma_B_complex = c9m.bendingNormalStress(M_B_complex, t)
print(
    f"B点由复杂弯曲引起的正应力 sigma_B_complex 为 {(sigma_0+sigma_B_complex)*1e-6 :.2f} MPa"
)

# 求解完毕！